In [1]:
import os
import sys
import subprocess
from skimage import io
import numpy as np
import itk
# Add the 'src' folder to the Python path
sys.path.append(os.path.abspath('../'))
from src.utils import match_h5_files_by_channels, setup_logging
from src.registration import register_and_transform
from src.stats import compute_region_stats
import logging

# single animal

In [3]:
def process_animal(animal, files, base_dir, fx, param_files, annotation_np):
    """
    Process a single animal by:
    - Registering and transforming the images
    - Computing regional statistics
    """
    output_dir = os.path.join(base_dir, animal, 'itk')  # Ensure logs and outputs go to /itk
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Step 1: Register and Transform
    logging.info("Starting registration and transformation.")
    register_and_transform(fx, files, output_dir, param_files)
    logging.info("Finished registration and transformation.")

    # Step 2: Compute Region Statistics
    logging.info("Starting computation of region statistics.")
    compute_region_stats(files, output_dir, annotation_np)
    logging.info("Finished computation of region statistics.")
    
    # Log the location of the logs and output files
    log_file = os.path.join(output_dir, f'registration_log_{animal}.txt')
    logging.info(f"Processing logs and outputs are saved in: {log_file}")

In [4]:
# Set the parameters manually here
animal = 'ANM550749_left_JF552'
base_dir = '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains'
fx_path = '/nrs/spruston/Boaz/I2/atlas10_hemi.tif'
param_files_dir = '/nrs/spruston/Boaz/I2/itk'
annotation_np_path = '/nrs/spruston/Boaz/I2/annotatin10_hemi.tif'

In [ ]:
setup_logging(base_dir, animal)

# Load the fixed image (fx)
logging.info(f"Loading fixed image (fx) from {fx_path}.")
fx = itk.imread(fx_path, pixel_type=itk.US)

# Load the parameter files
param_files = [
    os.path.join(param_files_dir, 'Order1_Par0000affine.txt'),
    os.path.join(param_files_dir, 'Order3_Par0000bspline.txt'),
    os.path.join(param_files_dir, 'Order4_Par0000bspline.txt'),
    os.path.join(param_files_dir, 'Order5_Par0000bspline.txt')
]
logging.info(f"Loaded parameter files from {param_files_dir}.")



In [ ]:
# Load the annotation volume
logging.info(f"Loading annotation volume from {annotation_np_path}.")
annotation_np = np.int64(io.imread(annotation_np_path))


In [ ]:
# Match H5 files by channels for all animals
logging.info(f"Matching H5 files in {base_dir}.")
animals_files = match_h5_files_by_channels(base_dir)

In [ ]:
# Get the files for the selected animal
files = animals_files.get(animal)
files

In [ ]:
process_animal(animal, files, base_dir, fx, param_files, annotation_np)
logging.info("Processing completed.")

# Multiple animals using bsub

In [ ]:
# Define parameters
base_dir = '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains'
fx = '/nrs/spruston/Boaz/I2/atlas10_hemi.tif'
param_files_dir = '/nrs/spruston/Boaz/I2/itk'
annotation_np = '/nrs/spruston/Boaz/I2/annotatin10_hemi.tif'
python_script = '../src/main.py'  
num_cores = 48


In [ ]:
logs_dir = '../logs'
# Create the logs directory if it doesn't exist
os.makedirs(logs_dir, exist_ok=True)

In [ ]:
animals = [
    "ANM550749_left_JF552",
    "ANM550751_left_JF673",
    "ANM551089_left_JF673",
    # Add more animal IDs here if needed
]

In [ ]:
for animal in animals:
    # Construct the log file paths
    output_log = os.path.join(logs_dir, f'output_{animal}.log')
    error_log = os.path.join(logs_dir, f'error_{animal}.log')

    # Construct the bsub command
    bsub_command = [
        'bsub',
        '-J', f'job_{animal}',  # Job name
        '-n', str(num_cores),  # Number of CPU cores
        '-o', output_log,  # Output log
        '-e', error_log,  # Error log
        'python', python_script, 
        '--animal', animal,
        '--base_dir', base_dir,
        '--fx', fx,
        '--param_files_dir', param_files_dir,
        '--annotation_np', annotation_np
    ]

    # Submit the job to the cluster
    print(f"Submitting job for animal: {animal}")
    subprocess.run(bsub_command)